In [4]:
from __future__ import print_function


import numpy as np
import theano
import theano.tensor as T
import lasagne
import urllib2

#Lasagne Seed for Reproducibility
lasagne.random.set_rng(np.random.RandomState(1))

# Sequence Length
SEQ_LENGTH = 64

# Number of units in the two hidden (LSTM) layers
N_HIDDEN = 512

# Optimization learning rate
LEARNING_RATE = .01

# All gradients above this will be clipped
GRAD_CLIP = 100

# How often should we check the output?
PRINT_FREQ = 10

# Number of epochs to train the net
NUM_EPOCHS = 200

# Batch Size
BATCH_SIZE = 512

files = ["Data/irish_music.txt", "Data/swedish_music.txt","Data/french_music.txt"]
context = ["irish","swedish","french"]
in_text=[0]*len(files)
for i in range(0,len(files)) :
    try:
        #You can also use your own file
        #The file must be a simple text file.
        #Simply edit the file name below and uncomment the line.
        #in_text = open('data.txt').read() #nietzsche text
        in_text[i] = open(files[i], 'r').read() #Music file
        in_text[i]= in_text[i].replace('  ','')
        in_text[i]= in_text[i].replace('\r','')
        in_text[i]=''.join([j if ord(j) < 128 else ' ' for j in in_text[i]])
        in_text[i] = in_text[i].decode("utf-8-sig").encode("utf-8")
    except Exception as e:
        print(e)
        print("Please verify the location of the input file/URL.")
        print("A sample txt file can be downloaded from https://s3.amazonaws.com/text-datasets/nietzsche.txt")
        raise IOError('Unable to Read Text')

#generation_phrase = "g2B2|d2d2 d2e2|d2cB g2fg|a2A2 A2\nGA|B2B2 cBAG|A2B2 g2fg|a2gf g2fe|d2B2 B2:|\n<end>\n<start>\n"
#generation_phrase="The quick brown fox jumps"
generation_phrases=[]
generation_phrases.append("a2 ba ge | d2 ed cB | c2 cd Bc | A4 AB |\ncB cd ef | g4 fe | d2 dB AG | E4 Bd |\ne2 ge dB | A2 B/c/d D2 | E2 GA/B/ A>G | G4 || \n<end>\n<start>\n")
generation_phrases.append("|:A2df a2a2 g4|A2^ce gage f3f|fafd ege^c d3e|f2ed ^c2ec A4|\nA2df a2a2 g4|A2^ce gage f3f|fafd ege^c d3e|f2ed ^c2ec d4:|\n<end>\n<start>\n")
generation_phrases.append("g2gb g2gb g2gb | f2fa f2fa f2fa | f2ed ^cde^c A4 :|\n|: A2d2 ^cde^c d4 | A2f2 efge f4 | A2d2 ^cde^c d4 |\nA2cA G2BG A4 | A2cA G2BG F2AF | EFE^C D8 :| \n<end>\n<start>\n")

a=set()
for text in in_text:
        a = a|set(text)
chars = list(a)
chars.sort()
data_size, vocab_size = sum(len(text) for text in in_text), len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
vocab_size = vocab_size+3
n = len(chars)
char_to_ix['irish']=n
char_to_ix['swedish']=n+1
char_to_ix['french']=n+2
ix_to_char[n]='irish'
ix_to_char[n+1]='swedish' 
ix_to_char[n+2]='french'

def gen_data(p, data,context, batch_size = BATCH_SIZE, return_target=True):
    '''
    This function produces a semi-redundant batch of training samples from the location 'p' in the provided string (data).
    For instance, assuming SEQ_LENGTH = 5 and p=0, the function would create batches of 
    5 characters of the string (starting from the 0th character and stepping by 1 for each semi-redundant batch)
    as the input and the next character as the target.
    To make this clear, let us look at a concrete example. Assume that SEQ_LENGTH = 5, p = 0 and BATCH_SIZE = 2
    If the input string was "The quick brown fox jumps over the lazy dog.",
    For the first data point,
    x (the inputs to the neural network) would correspond to the encoding of 'T','h','e',' ','q'
    y (the targets of the neural network) would be the encoding of 'u'
    For the second point,
    x (the inputs to the neural network) would correspond to the encoding of 'h','e',' ','q', 'u'
    y (the targets of the neural network) would be the encoding of 'i'
    The data points are then stacked (into a three-dimensional tensor of size (batch_size,SEQ_LENGTH,vocab_size))
    and returned. 
    Notice that there is overlap of characters between the batches (hence the name, semi-redundant batch).
    '''
    x = np.zeros((batch_size,SEQ_LENGTH,vocab_size))
    y = np.zeros(batch_size)

    for n in range(batch_size):
        ptr = n
        for i in range(SEQ_LENGTH):
            x[n,i,char_to_ix[data[p+ptr+i]]] = 1.
            x[n,i,char_to_ix[context]] = 1.
        if(return_target):
            y[n] = char_to_ix[data[p+ptr+SEQ_LENGTH]]
    return x, np.array(y,dtype='int32')

def getTune(context,N=600):
    if context == 'irish':
        generation_phrase = generation_phrases[0]
    elif context == 'swedish':
        generation_phrase = generation_phrases[1]
    else:
        generation_phrase = generation_phrases[2]
    sample_ix = []
    x,_ = gen_data(len(generation_phrase)-SEQ_LENGTH,generation_phrase,context,1,0)

    for i in range(N):
            # Pick the character that got assigned the highest probability
            ix = np.argmax(probs(x).ravel())
            # Alternatively, to sample from the distribution instead:
            # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
            sample_ix.append(ix)
            x[:,0:SEQ_LENGTH-1,:] = x[:,1:,:]
            x[:,SEQ_LENGTH-1,:] = 0
            x[0,SEQ_LENGTH-1,sample_ix[-1]] = 1. 

    random_snippet = ''.join(ix_to_char[ix] for ix in sample_ix)
    return random_snippet

import pickle

print("Building network ...")
   
# First, we build the network, starting with an input layer
# Recurrent layers expect input of shape
# (batch size, SEQ_LENGTH, num_features)

l_in = lasagne.layers.InputLayer(shape=(None, None, vocab_size))

# We now build the LSTM layer which takes l_in as the input layer
# We clip the gradients at GRAD_CLIP to prevent the problem of exploding gradients. 

l_forward_1 = lasagne.layers.LSTMLayer(
        l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
        nonlinearity=lasagne.nonlinearities.tanh)

l_forward_1_drop = lasagne.layers.DropoutLayer(l_forward_1, p=0.1)
    
l_forward_2 = lasagne.layers.LSTMLayer(
        l_forward_1_drop, N_HIDDEN, grad_clipping=GRAD_CLIP,
        nonlinearity=lasagne.nonlinearities.tanh)

# The l_forward layer creates an output of dimension (batch_size, SEQ_LENGTH, N_HIDDEN)
# Since we are only interested in the final prediction, we isolate that quantity and feed it to the next layer. 
# The output of the sliced layer will then be of size (batch_size, N_HIDDEN)
l_forward_slice = lasagne.layers.SliceLayer(l_forward_2, -1, 1)

# The sliced output is then passed through the softmax nonlinearity to create probability distribution of the prediction
# The output of this stage is (batch_size, vocab_size)
l_out = lasagne.layers.DenseLayer(l_forward_slice, num_units=vocab_size, W = lasagne.init.Normal(), nonlinearity=lasagne.nonlinearities.softmax)

# Theano tensor for the targets
target_values = T.ivector('target_output')
    
# lasagne.layers.get_output produces a variable for the output of the net
network_output = lasagne.layers.get_output(l_out)

# The loss function is calculated as the mean of the (categorical) cross-entropy between the prediction and target.
cost = T.nnet.categorical_crossentropy(network_output,target_values).mean()

# Retrieve all parameters from the network
params = pickle.load(open("params_context_country_3","rb"))
lasagne.layers.set_all_param_values(l_out,params)
all_params = lasagne.layers.get_all_params(l_out,trainable=True)

# Compute AdaGrad updates for training
print("Computing updates ...")
updates = lasagne.updates.adagrad(cost, all_params, LEARNING_RATE)

# Theano functions for training and computing cost
print("Compiling functions ...")
train = theano.function([l_in.input_var, target_values], cost, updates=updates, allow_input_downcast=True)
compute_cost = theano.function([l_in.input_var, target_values], cost, allow_input_downcast=True)

# In order to generate text from the network, we need the probability distribution of the next character given
# the state of the network and the input (a seed).
# In order to produce the probability distribution of the prediction, we compile a function called probs. 
    
probs = theano.function([l_in.input_var],network_output,allow_input_downcast=True)


Building network ...
Computing updates ...
Compiling functions ...


In [ ]:
from bottle import route,get, run,static_file,template 
from music21 import *
from itertools import count
import base64

iid = count()

@route('/irish2')
def irish():
    try:
        tune = getTune('irish')
        if '<end>' in tune:
            tune = (tune.split("<end>"))[0]
        s = converter.parse(tune)
        mf = midi.translate.streamToMidiFile(s)
        string="data:audio/mid;base64,"
        id_tune = next(iid)+1
        f=open('/home/ubuntu/Project/Recipes/examples/CSE253/midi/irish'+str(id_tune)+'.mid','w')
        f.write(string)
        f.close()
        out = open("/home/ubuntu/Project/Recipes/examples/CSE253/midi/irish"+str(id_tune)+".mid",'ab')
        out.write(base64.b64encode(mf.writestr()))
        out.close()
        return 'irish'+str(id_tune)+'.mid@@@@'+tune
    except Exception as e:
        print(e)
        out =  open('/home/ubuntu/Project/Recipes/examples/CSE253/log/log.txt','a')
        out.write(str(e)+"\n")
        out.close()
        return 'irish.mid@@@@'+tune

@route('/swedish2')
def swedish():
    try:
        tune = getTune('swedish')
        if '<end>' in tune:
            tune = (tune.split("<end>"))[0]
        s = converter.parse(tune)
        mf = midi.translate.streamToMidiFile(s)
        string="data:audio/mid;base64,"
        id_tune = next(iid)+1
        f=open('/home/ubuntu/Project/Recipes/examples/CSE253/midi/swedish'+str(id_tune)+'.mid','w')
        f.write(string)
        f.close()
        out = open("/home/ubuntu/Project/Recipes/examples/CSE253/midi/swedish"+str(id_tune)+".mid",'ab')
        out.write(base64.b64encode(mf.writestr()))
        out.close()
        return 'swedish'+str(id_tune)+'.mid@@@@'+tune
    except Exception as e:
        print(e)
        out =  open('/home/ubuntu/Project/Recipes/examples/CSE253/log/log.txt','a')
        out.write(str(e)+"\n")
        out.close()
        return 'swedish.mid@@@@'+tune

@route('/french2')
def french():
    try:
        tune = getTune('french')
        if '<end>' in tune:
            tune = (tune.split("<end>"))[0]
        s = converter.parse(tune)
        mf = midi.translate.streamToMidiFile(s)
        string="data:audio/mid;base64,"
        id_tune = next(iid)+1
        f=open('/home/ubuntu/Project/Recipes/examples/CSE253/midi/french'+str(id_tune)+'.mid','w')
        f.write(string)
        f.close()
        out = open("/home/ubuntu/Project/Recipes/examples/CSE253/midi/french"+str(id_tune)+".mid",'ab')
        out.write(base64.b64encode(mf.writestr()))
        out.close()
        return 'french'+str(id_tune)+'.mid@@@@'+tune
    except Exception as e:
        print(e)
        out =  open('/home/ubuntu/Project/Recipes/examples/CSE253/log/log.txt','a')
        out.write(str(e)+"\n")
        out.close()
        return 'french.mid@@@@'+tune    
    
@route('/tunes')
def serve_homepage():
    return static_file('index.html', root='/home/ubuntu/Project/Recipes/examples/CSE253')

@route('/<filename:path>')
def send_static(filename):
    return static_file(filename, root='/home/ubuntu/Project/Recipes/examples/CSE253') 

run(host='ec2-54-67-49-59.us-west-1.compute.amazonaws.com', port=8000, debug=True)


Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://ec2-54-67-49-59.us-west-1.compute.amazonaws.com:8000/
Hit Ctrl-C to quit.

/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:96: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:675: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:667: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/formatters.py:670: DeprecationWarning: PlainTextFormatter._type_printers_default is dep